# (day4) Section 6: 物体検知 - セグメンテーション

本書は、「深層学習後編（day4）レポート」の、「Section 6: 物体検知 - セグメンテーション」についてのものです。


## 1. 要点まとめ


<!-- - page. 124

### Introduction
#### 広義の物体認識タスク
### 代表的データセット
### 評価指標
#### 分類問題における評価指標
#### 閾値変化に対する振る舞い
#### IOU: Intersection over Union
#### Precision/Recall
#### AP: Average Precision
#### mAP mean Average Precision
#### FPS: Flames per Second
### 物体検知の大枠
#### マイルストーン: 深層学習以降の物体検知
#### 物体検知のフレームワーク
#### One-stage detector, Two-stage detector
### SSD: Single Shot Detector
#### SSD in a nutshell
#### VGG16 のネットワーク図
#### SSD のネットワークアーキテクチャ
#### 特徴マップからの出力
#### SSD のデフォルトボックス数
#### その手の工夫
#### 損失関数
### Semantic Segmentation の概略
#### 要点
#### VCG16
#### Up-samplng と Pooling
#### NN における受容野
#### FCN ( Fully Convolutional Network ) の基本
#### Deconvolution / Transposed convolution
#### 輪郭情報の補完
#### U-Net
#### DeconvNet & SegNet
#### Unpooling
#### Dilated Convolution -->


#### 広義の物体認識タスク

物体認識タスクの種類は、以下の通り。

| 種類 | 出力 |
| -- | -- |
| 分類 | 画像1つの中のクラスラベル |
| 物体検知 | Bounding Box |
| 意味領域分割 | 各ピクセルのクラスラベル |
| 個体領域分割 | 各ピクセルのクラスラベル |


### 代表的データセット

物体検知の性能を評価する上で、基準となるデータセットが重要。


| データセット | クラス | Train + Val | Box / 画像 | Annotation |
| -- | -- | -- | -- | -- |
| VOC12 | 20 | 11,540 | 1.1 | Instance Annotation |
| ILSVRC17 | 200 | 476,668 | 2.4 | |
| MS COCO18 | 80 | 123,287 | 7.3 | Instance Annotation |
| OICOD18 | 500 | 1,743,042 | 7.0 | Instance Annotation |

クラス数が大きいことが有利であるわけではない。
( 意味の少ない分類も多い )


### 評価指標



#### 分類問題における評価指標



##### Confusion Matrix

| 真値 - 予測 | Positive | Negative |
| -- | -- | -- |
| Positive | True<br/>Positve | False<br/>Negative |
| Negative | False<br/>Positve | True<br/>Negative |

$
Precision = \frac{TP}{TP + FP}
$

$
Recall = \frac{TP}{TP + FN}
$

confidence の閾値を変化させて、 Precision-Recall curve を描く。


#### 閾値変化に対する振る舞い

- クラス分類

- 物体検出<br/>
  閾値 ( conf. の Threashold ) を上げると、
  クラス分類の場合、サンプルの数は変わらない。
  物体検出の場合、認識される数 ( BBox の数 ) 自体が減る。

<!-- - TODO/Done: Precision-Recall curve との関連。 -->


#### IOU: Intersection over Union

クラスラベルと物体位置の予測精度を評価するためのもの。

Confusion Matrix の要素から計算。

$$
IoU = \frac{Area \, of \, Overlap}{Area \, of \, Union} \\
= \frac{TP}{TP + FP + FN}
$$

- NOTE: TN は、評価の範囲外なので、分母に含まれない。

Jaccard 係数とも言う。

- [Intersection over Union (IoU) for object detection - PyImageSearch](https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/)


#### Precision/Recall

$ conf \geq 0.5, IoU \geq 0.5 $ が評価対象。

$ IoU \gt Threashold $ で、 TP 、$ \leq $ ならば FP

- 入力 1 枚に対しての評価<br/>
  複数のクラスを評価。<br/>
  既に検知済みのクラスを重複して検知した場合は、 ( 検知が正しいとしても ) FP とする。

- クラス単位の評価<br/>
  画像は複数で、あるクラスについて評価。 ( 講義資料の例では、「人」 1 クラス )<br/>
  ここでも既に検出済みは、 FP 。<br/>
  また、 Recall において、 1 回も検出されなかった Ground-Truth の画像は、 FN で計数。


<!-- - TODO/Done: クラス単位の FN 検知に対応する表の行? -->


#### AP: Average Precision

対象のクラスラベルを固定にして、

conf. の閾値を $ \beta $ とするとき、

$$
Recall = R(\beta), Precision = P(\beta)
$$

Precision-Recall curve は、

$$
P = f(R)
$$

Average Precision は、

$$
AP
=
\int_{0}^{1} P(R) d R
$$

Precision-Recall curve の PB 曲線の、
下側面積を意味する。


#### mAP mean Average Precision

クラスごとに計算された AP の算術平均。

クラス数が C のとき、

$$
mAP = \frac{1}{C} \sum_{i=1}^C AP_i
$$


#### FPS: Flames per Second

検出速度の指標。

横軸を FPS 、縦軸を AP にして、プロットする。

FPS を大きく ( 検知速度を速める ) すると、
精度 ( AP ) が低下する傾向にある。


### 物体検知の大枠


#### マイルストーン: 深層学習以降の物体検知

- 2012 AlexNet<br/>
  SIFT: Scale Invariant Feature Transform から、<br/>
  DCNN: Deep Convolutional Neural Network が主流に。

- [元論文](https://proceedings.neurips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf)


#### 物体検知のフレームワーク

以下の 2 種類。近年は、 One-stage detector が主流。

| 種類 | 候補領域の検出</br>クラス推定 | 精度 | 計算量 | 推論 | 
| -- | -- | -- | -- | -- |
| Two-stage detector | 別々に行う | 相対的に高い | 相対的に大きい | 遅い |
| One-stage detector | 同時に行う | 相対的に低い | 相対的に小さい | 早い |

Two-stage detector では、特徴マップから候補領域の検出のネットワークを経由することに対して、分類のネットワークに至る野に対して、<br/>
One-stage detector では、1 つの Detection ネットワークで構成される。


### SSD: Single Shot Detector

具体的な One-stage detector のアルゴリズムの一つとして、 SSD がある。

<!-- - TODO/Done: SSD は、 One-stage detector のことか? -->


#### SSD in a nutshell

1. 適当な位置、大きさの、Default BOX を用意して、検出したい物体を囲む。 

1. Default BOX を検出したい物体に合せるように変形して、 Conf. を出力する。<br/>
  この繰り返しで、学習を進める。


#### VGG16 のネットワーク図

- [原著論文](https://arxiv.org/pdf/1409.1556.pdf)<br/>
  > 2.1 ARCHITECTURE

CN と FN の合計で、16層のアーキテクチャ。


#### SSD のネットワークアーキテクチャ

- [原著論文](https://arxiv.org/pdf/1512.02325.pdf)<br/>
  以下の、上の図。<br/>
  > 2.1 Model<br/>
  ...<br/>
  > Fig. 2: A comparison between two single shot detection models: SSD and YOLO

- SSD は、 VGG16 を元に構成。<br/>
  >  We use the VGG-16 network as a base, but other networks should also produce good results

- 入力サイズにより、 SSD300、SSD512 と表記。<br/>
  上の図は、 SSD300 。<br/>
  > Figure 2 shows the architecture details of the SSD300 model.

- VGG16 の FC ( "three Fully-Connected (FC) layers" の 2 層 ) を Conv 層に変更。<br/>
  一番最後の FN も削除。

- VGG16 の10層目に相当する Conv 層

- SSD の特徴であるマルチスケール特徴マップ</br>
  > multiple feature maps with different resolutions


#### 特徴マップからの出力

引用元の以下。

> 2 The Single Shot Detector (SSD)<br/>
...<br/>
> Fig. 1: SSD framework.

> 2.1 Model<br/>
...<br/>
> Default boxes and aspect ratios

- 出力サイズ `#Class + 4` の、オフセット項

オフセット項は、 $ \Delta x, \Delta y, \Delta w, \Delta h $

$$
x = x_0 + 0.1 \times \Delta x \times w_0 \\
y = y_0 + 0.1 \times \Delta y \times w_0 \\
w = w_0 + 0.1 \times \Delta w \times w_0 \\
h = h_0 + 0.1 \times \Delta h \times w_0
$$

特徴マップの各特徴量に、 k 個の Default Box を用意、<br/>
さらに、特徴マップのサイズを、 m x n とすると、<br/>
特徴マップの出力サイズは、 k x ( クラス数 + 4 (オフセット項の数) )


#### SSD のデフォルトボックス数

- VGG-16 through Conv5 3 layer からの Default Bbox は、各特徴に 4 つ。

- Classifier 以降の Default Bbox は、各特徴に 6 つ。<br/>
  これは計算量の都合のため、 Default Box の数を調整したもの。

> 3.1 PASCAL VOC2007<br/>
> For conv4 3,
conv10 2 and conv11 2, we only associate 4 default boxes at each feature map location<br/>
...<br/>
>  For all other layers, we put 6 default boxes as
described in Sec. 2.2.

各ネットワークの特徴マップ・サイズに、 ( VOC データセット ) クラス数 20 + 背景 1 として、
`( #Class = 21 ) + 4` を掛けたものが、デフォルトボックスの数になる。


- アーキテクチャの図で、特徴マップのサイズで図の要素の大きさを表現している。<br/>
  しかし、この大きさ ( 特徴マップのサイズ ) と、解像度は別である。<br/>
  解像度が大きい層は、小さな物体を認識し、
  解像度が小さい層は、大きな物体を認識する。


### その他の工夫

- non-maximum suppression (nms)<br/>
  複数の BBox が重なってしまっている場合への対応。<br/>
  IoU が同じであれば、同じ対象を選択している可能性が高いため、 Conf. の高い BBox を残す。<br/>
  > By using a confidence threshold of 0.01, we can filter out most boxes. 

- Hard negative mining<br/>
  ネガティブな BBox を消す。<br/>
  学習時にデフォルトボックスをフィルター。<br/>
  > Instead of
using all the negative examples, we sort them using the highest confidence loss for each
default box and pick the top ones so that the ratio between the negatives and positives is
at most 3:1.

その他に、重ならない様に default bonding box の位置と縦横比を決める、
"Choosing scales and aspect ratios for default boxes" 、<br/>
学習でより多様な訓練データを得るための、
"Data augmentation" がある。


#### 損失関数

引用元の以下の箇所。

> 2.2 Training<br/>
...<br/>
> Training objective

confidence の損失と、検出位置の損失の、和。

> The overall objective loss
function is a weighted sum of the localization loss (loc) and the confidence loss (conf):

- NOTE: [参考引用: Smooth L1 Loss](https://arxiv.org/pdf/1504.08083.pdf)<br/>
  > 2.3. Fine-tuning for detection<br/>
  ...<br/>
  > Multi-task loss.

$$
smooth_{L1}(x) = \begin{equation}
\begin{cases}
0.5x^2 & if \; x \gt 0 \\
|x| - 0.5 & otherwise,
\end{cases}
\end{equation}
$$


### Semantic Segmentation の概略


#### 要点

順伝播で推論、逆伝播で学習。
Convolution + Pooling というとろは今までと同様。<br/>
ただ、要約したクラス数から、分割した領域のサイズに復元するために、 "Up-sampling" の壁がある。

例えば、 VGG16 の場合、 ( 最後の ) max pooling によって、 7 x 7 の解像度まで下がる。


#### Up-samplng と Pooling - NN における受容野 -

Pooling はある程度の大きさのある受容野を確保するためのもの。

受容野を大きくするために、深い Conv 層と pooling + stride で対応、
深い Conv 層は計算資源的に効率的でない。
なので、 pooling + stride が受容野を広げるために必要。


#### FCN ( Fully Convolutional Network ) の基本

VGG16 の最後の FNN を Conv 層に置き換えている。

- [引用元](https://arxiv.org/pdf/1411.4038.pdf)<br/>
  > Figure 2. Transforming fully connected layers into convolution
layers enables a classification net to output a heatmap.

"tabby cat" という特徴のヒートマップを得る。
このヒートマップを up sampling で元の解像度に戻して、領域における分類を行う。

<!-- - TODO/Done: 結論までの論理的な道筋が不十分。 -->


#### Deconvolution / Transposed convolution

- [vdumoulin/conv_arithmetic: A technical report on convolution arithmetic in the context of deep learning](https://github.com/vdumoulin/conv_arithmetic)<br/>
  各種類の、畳み込みの様子のアニメーション。
  - 対応するアニメーションは、<br/>
    - [Transposed convolution animations](https://github.com/vdumoulin/conv_arithmetic#transposed-convolution-animations)<br/>
      "Padding, strides, transposed"


- NOTE: DECONVOLUTION<br/>
  - https://arxiv.org/pdf/1905.11926.pdf<br/>
    page. 2 の馬の画像の例。


#### 輪郭情報の補完

Pooling すると輪郭が失われていく。 ( ぼやける )
これを低レイヤーの出力で保管する。
( 低レイヤーの出力を element wise addition: 要素ごとに追加する )

- [引用元](https://arxiv.org/pdf/1411.4038.pdf)<br/>
  > Figure 3. Our DAG nets learn to combine coarse, high layer information with fine, low layer information.

FFCN-8s でだいぶ輪郭が復元できていることが分かる。


#### U-Net

<!-- - TODO/Done: U-Net とは? 特徴? -->

FCN と類似して、低レイヤーの出力を使用する例として、 U-Net がある。

- [出典](https://arxiv.org/pdf/1505.04597.pdf)<br/>
  > Fig. 1. U-net architecture

Skip-connection として pooling 前の情報を使う。

FCN の element wise addition と違い、チャネル方向に低レイヤーの出力を結合している。


#### DeconvNet & SegNet

Semantic Segmentaion のネットワークの例。

どのネットワークも、解像度を低くして特徴を得た後、 up sampling で元の解像度に戻して個体領域分割の結果を得ている。

- [Learning Deconvolution Network for Semantic Segmentation](https://arxiv.org/pdf/1505.04366.pdf)<br/>
  > The deconvolution network is composed of deconvolution and unpooling layers, which identify pixel-wise
class labels and predict segmentation masks.

- [SegNet: A Deep Convolutional
Encoder-Decoder Architecture for Image
Segmentation](https://arxiv.org/pdf/1511.00561.pdf)<br/>
  > 

<!-- - TODO/Done: ここも各 Net の特徴など聞く。 -->


#### Unpooling

Deconvolution と異なったアプローチの up sampling 方法。

Pooling 時に位置情報を保持しておき、
up sampling の時に、保持していた位置情報を使う。
位置情報と言うのは、例えば Max pooing ならば、どこが最大だったかの位置のこと。
どこが最大だったかの情報を使って、 up sampling できる。

- 引用元の、<br/>
  > 3.2.1 Unpooling

Pooling により、受容野の空間的な情報が失われる。
これを元のサイズに再構築するのが、 Unpooling 層。



#### Dilated Convolution

pooling ではなく、 Conv 層で受容野を広げる手法。

- [MULTI-SCALE CONTEXT AGGREGATION BY
DILATED CONVOLUTIONS](https://arxiv.org/pdf/1511.07122.pdf)<br/>
  > Figure 1: Systematic dilation supports exponential expansion of the receptive field without loss of
resolution or coverage.

$ i + 1 $ 番目の層のサイズを、以下 $ F_{i+1} $ に拡張できる。

$ F_{i+1} = (2^{i+2} - 1) \times (2^{i+2} - 1) $


<!-- - TODO: 7 層必要になる計算の方法 -->
